In [49]:
import re, json

In [296]:
import json

def simpan(data,nama):
    json.dump(data,open(nama+'.json','w'))
def buka(nama):
    data = json.load(open(nama+'.json','r'))
    return data

stopWord = buka('../app/static/stopwords-id')
sinonim = buka('../app/static/sinonim')
soal = buka('../app/static/soal')
jawaban = buka('../app/static/jawaban')

soal = soal[len(soal)//2:]
jawaban = jawaban[len(jawaban)//2:]

#Menghitung border function dari string s
def computeFail(s):
    fail = [0 for i in range(len(s))]
    fail[0] = 0
    m = len(s)
    i, j = 1, 0
    while(i < m):
        if(s[j] == s[i]):
            fail[i] = j+1
            i += 1
            j += 1
        elif (j > 0):
            j = fail[j-1]
        else:
            fail[i] = 0
            i += 1
    return fail

#Mencocokan pattern di text (apakah pattern ada di text)
#Merupakkan exact match
#Algoritma sesuai slide matkul Stima 2019 dengan modifikasi
#agar bisa mengembalikan panjang kata terpanjang yang mirip
#O(m+n)
#Return : [index, maxLen kata yang mirip]
def kmpITB(text, pattern):
    #Hitung border function
    n = len(text)
    m = len(pattern)
    fail = computeFail(pattern)
    i, j = 0, 0
    maxLen = 0
    while(i < n):
        if(pattern[j] == text[i]):
            if(j == m-1):
                return [i-m+1, j+1] #Ketemu
            i += 1
            j += 1
        else:
            if(maxLen < j):
                maxLen = j
            if (j > 0):
                j = fail[j-1]
            else:
                i += 1
    return [-1, maxLen] #Tidak ketemu

#Menghitung last occurence untuk boyer-moore dari string s
def computeLast(s):
    last = [-1 for i in range(128)]
    for i in range(len(s)):
        last[ord(s[i])] = i
    return last

#Mencocokan pattern di text
#Merupakan exact match
#Algoritma sesuai slide matkul Stima 2019 dengan modifikasi
#agar bisa mengembalikan panjang kata terpanjang yang mirip
#O(nm + A), A = banyak alphabet
#Return : [index, maxLen kata yang mirip]
def bmITB(text, pattern):
    last = computeLast(pattern)
    n, m = len(text), len(pattern)
    i = m-1
    if(i > n-1):
        return [-1,0]
    j = m-1
    maxLen = 0
    while(i <= n-1):
        if(pattern[j] == text[i]):
            if(j == 0):
                return [i, n-j]
            else:
                i -= 1
                j -= 1
        else:
            if(maxLen < n-j):
                maxLen = n-j
            lo = last[ord(text[i])]
            i = i + m - min(j, 1+lo)
            j = m-1
    return [-1, n-maxLen]

def hapusStopWord(s):
    tempKata = s.split(' ')
    mark = [False for i in tempKata]
    for sw in stopWord:
        while(sw in tempKata):
            tempKata.remove(sw)
    return ' '.join(tempKata)

def hitungCocok(text, pattern):
    text = text.lower()
    pattern = pattern.lower()
    if(kmpITB(pattern, text)[0] > -1):
        return 1
    else:
        totalCocok = 0
        #Cocokkan per kata (Boleh sesuai spek)
        for tS in text.split(' '):
            tS = tS.strip()
            if(tS != ''):
                maxCocok = 0
                for p in pattern.split(' '):
                    for s in getSinonim(p):
                        #print("Match ", s, ", ", t)
                        hasKMP = kmpITB(s, tS)
                        hasBM = bmITB(s, tS)
                        has = hasKMP
#                         if(hasKMP[1] > hasBM[1]):
#                             has = hasKMP
                        if(maxCocok < has[1]):
                            maxCocok = has[1]
                        if(has[0] > -1):
                            break
                if(maxCocok != 0):
                    totalCocok += 1
                totalCocok += maxCocok
        return min(totalCocok/len(text), 1)
    
def getSinonim(kata):
    hasil = {kata}
    if(kata in sinonim):
        if('sinonim' in sinonim[kata]):
            hasil = hasil.union(set(sinonim[kata]['sinonim']))
#     for s in sinonim:
#         if(kata in sinonim[s]['sinonim']):
#             hasil = hasil.union(set([s]))
#     for h in hasil:
#         if(kmpITB(kata, h) == -1 and kmpITB(h, kata) == -1):
#             hasil.union(getSinonim(h))
    return hasil

def getJawaban(pertanyaan):
    pertanyaan = hapusStopWord(pertanyaan)
    mirip = []
    for idx, s in enumerate(soal):
        s = hapusStopWord(s)
        h = hitungCocok(s, pertanyaan)
        if(h >= 0.5):
            mirip.append([h, jawaban[idx]])
        if(h == 1):
            break
    mirip.sort(key = lambda x : x[0], reverse=True)
    if(len(mirip) > 0):
        if(mirip[0][0] == 1):
            return mirip[:1]
    return mirip[:3]

In [214]:
hitungCocok('can you tell me what is exactly python programming language', 'what is programming language')

1

In [263]:
hitungCocok('anjing', 'anjig wow')

0.8333333333333334

In [261]:
bmITB('anjiaag wow', 'anjing wow')

[-1, 4]

In [262]:
kmpITB('anjiaag wow', 'anjing wow')

[-1, 4]

In [298]:
getJawaban('what is python')

[[1,
  'Python is an interpreted, interactive, object-oriented programming language. It incorporates modules, exceptions, dynamic typing, very high level dynamic data types, and classes. Python combines remarkable power with very clear syntax. It has interfaces to many system calls and libraries, as well as to various window systems, and is extensible in C or C++. It is also usable as an extension language for applications that need a programmable interface. Finally, Python is portable: it runs on many Unix variants, on the Mac, and on Windows 2000 and later.\nTo find out more, start with The Python Tutorial. The Beginner’s Guide to Python links to other introductory tutorials and resources for learning Python.\n']]

In [297]:
soal

['Saya merupakan lulusan salah satu program D3 di Indonesia. Apakah saya dapat melanjutkan program S1 ke ITB ',
 'Apakah ITB menyediakan perkuliahan jarak jauh atau perkuliahan secara online ',
 'Apakah ITB menyediakan kelas karyawan atau kelas kuliah malam ',
 'Apakah ITB menyediakan program D3 ',
 'What is Python',
 'What is the Python Software Foundation',
 'Are there copyright restrictions on the use of Python',
 'Why was Python created in the first place',
 'What is Python good for',
 'How does the Python version numbering scheme work',
 'How do I obtain a copy of the Python source',
 'How do I get documentation on Python',
 'I’ve never programmed before. Is there a Python tutorial',
 'Is there a newsgroup or mailing list devoted to Python',
 'How do I get a beta test version of Python',
 'How do I submit bug reports and patches for Python',
 'Are there any published articles about Python that I can reference',
 'Are there any books on Python',
 'Where in the world is www.python.o

In [267]:
soal[12]

'Siapa pemain catur nomor satu dunia sekarang'

In [283]:
s = hapusStopWord(soal[12].lower())

In [284]:
s

'pemain catur nomor dunia'

In [292]:
text = 'what is python'
totalCocok = 0
pattern = s
for tS in text.split(' '):
    tS = tS.strip()
    maxCocok = 0
    if(tS != ''):
        for p in pattern.split(' '):
            for s in getSinonim(p):
                hasKMP = kmpITB(s, tS)
                hasBM = bmITB(s, tS)
                has = hasKMP
#                 if(hasKMP[1] > hasBM[1]):
#                     has = hasKMP
                if(maxCocok < has[1]):
                    print("Match ", s, ", ", tS)
                    print('HAS = ', has[1])
                    maxCocok = has[1]
                if(has[0] > -1):
                    break
        if(maxCocok != 0):
            totalCocok += 1
        totalCocok += maxCocok
        print(totalCocok)

0
Match  dunia ,  is
HAS =  1
2
2


In [291]:
kmpITB('tanah', 'what')

[-1, 0]